# **Dependencies**

In [ ]:
# Starting library
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# nltk dependencies
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Dataset
slang_url = 'https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv'
dataset_url = 'https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/re_dataset.csv'

df_slang = pd.read_csv(slang_url, encoding = 'latin1', header = None)
df_slang.columns = ['original', 'swap']
df = pd.read_csv(dataset_url, encoding = 'latin1')

In [ ]:
# Helper function 
def check(word):
  return df[df['Tweet'].str.contains(word, case = False)]['Tweet'].sample(10).unique()

# Words removal function
def remove_words(sentence, word_list):
  word_tokens = word_tokenize(sentence)
    
  filtered = [w for w in word_tokens if w.lower() not in word_list]
    
  new = ' '.join(filtered)
    
  return new

# Words replace function
def replace_words(sentence, dictionary):
  word_list = dictionary.keys()
  word_tokens = word_tokenize(sentence)

  replaced = [dictionary[w] if w.lower() in list(dictionary.keys()) else w for w in word_tokens]

  new = ' '.join(replaced)

  return new

In [ ]:
# Slang words 
df_slang.head()

original                      swap
0  anakjakartaasikasik  anak jakarta asyik asyik
1         pakcikdahtua         pak cik sudah tua
2       pakcikmudalagi         pak cik muda lagi
3          t3tapjokowi              tetap jokowi
4                   3x                 tiga kali

In [ ]:
# Dataset
df.head()

Tweet  HS  Abusive  \
0  - disaat semua cowok berusaha melacak perhatia...   1        1   
1  RT USER: USER siapa yang telat ngasih tau elu?...   0        1   
2  41. Kadang aku berfikir, kenapa aku tetap perc...   0        0   
3  USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...   0        0   
4  USER USER Kaum cebong kapir udah keliatan dong...   1        1   

   HS_Individual  HS_Group  HS_Religion  HS_Race  HS_Physical  HS_Gender  \
0              1         0            0        0            0          0   
1              0         0            0        0            0          0   
2              0         0            0        0            0          0   
3              0         0            0        0            0          0   
4              0         1            1        0            0          0   

   HS_Other  HS_Weak  HS_Moderate  HS_Strong  
0         1        1            0          0  
1         0        0            0          0  
2         0        0            0          0  
3         0        0            0          0  
4         0        0            1          0

# **Cleansing**

In [ ]:
# Explore, sampling
df['Tweet'].sample(20).unique()

array(['USER Mulut Allah SWT di quran memang BUSUK, kafir, laknat dll',
       "USER USER Taplak meja warung sebelah, ya Allah humorku.'",
       'USER USER Baru tau ada presiden ekonomi he he emang yang milih menko anda ya',
       "USER Hdh cakep2 bolot'",
       "Kaum 1/2 kapir dengan #OtakJunkFood nya masih berkelit mencatut islam dg kegiatan2 yang merusak aqidah islam'",
       "USER USER betul bro.. PSI ini isinya sontoloyo semua bro.. yang jelas salah malah dilindungi dan diushakan menjadi benar..'",
       'Sebagai negara maju, ternyata segini gaji presiden AS dan pegawai pemerintahannya',
       'klik', 'Warisan, tradisi dan budaya mana?',
       "USER ohh brarti lu kemaren lupa sarapan, mangkanya sarap \\xf0\\x9f\\x98\\x82'",
       "USER USER Nah betul, klao sy si fair baik soeharto maupun soekarno memang pernah ada sala ketika mereka terlalu dengerin org yg muja2 mereka. Kalau soekarno terlalu dengerin pki, soeharto dengerin golkar'",
       'USER USER USER USER "lu gaada a

### Regex cleaning

In [ ]:
# Cleaning text using regex
def clean_text(Tweet):
    Tweet = re.sub('rt\s', '', Tweet)   # remove "rt" -> "retweet"
    Tweet = re.sub('x[\w]+', '', Tweet)    # remove xf, xa etc.
    Tweet = re.sub('[^a-zA-Z]+', ' ', Tweet)  # remove non-alphabet
    Tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', Tweet)   #remove link elements
    Tweet = re.sub('wk+', '', Tweet)    # remove "wkwk" -> indonesian laughing words
    Tweet = re.sub('[akwjhz]{3,}', '', Tweet)     # remove multiple occuring single alphabet noise "aaaa" "hhhh" etc.
    return Tweet

### Slang words

In [ ]:
# Create dictionary from slang words df
slang_dict = pd.Series(df_slang['swap'].values, index = df_slang['original']).to_dict()
for i in list(slang_dict.keys())[:10]:
  print(i, ':', slang_dict[i])

anakjakartaasikasik : anak jakarta asyik asyik
pakcikdahtua : pak cik sudah tua
pakcikmudalagi : pak cik muda lagi
t3tapjokowi : tetap jokowi
3x : tiga kali
aamiin : amin
aamiinn : amin
aamin : amin
aammiin : amin
abis : habis


In [ ]:
# Test replace words function
test_sen = "abis ini kita bkal pegi ke pasar"
replace_words(test_sen, slang_dict)

'habis ini kita bakal pergi ke pasar'

### Stop words

In [ ]:
# Stop words
stop_words = stopwords.words('indonesian')

# Stop words sample
stop_words[:10]

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir']

In [ ]:
# Add more words to remove

other_words = ['si', 'an', 'amp','user', 'pengguna', 'url', 'ru', 'uniform', 'resource', 'locator' ,'ya', 'iya', 'sih', 'nya', 'nih', 'banget', 'kayak']

character = [chr(i) for i in range(ord('a'), ord('z') + 1)]

stop_words_join = stop_words + other_words + character

In [ ]:
# Test remove stop word function
test_sen = "saya mau pergi ke sekolah naik mobil balap"
new_sen = remove_words(test_sen, stop_words)
new_sen

'pergi sekolah mobil balap'

### Cleaning pipeline

In [ ]:
# Tweet cleaning pipeline 
df['Tweet'] = df['Tweet'].str.lower()
df['Tweet'] = df['Tweet'].apply(lambda x : clean_text(x))
df['Tweet'] = df['Tweet'].apply(lambda x : replace_words(x, slang_dict))
df['Tweet'] = df['Tweet'].apply(lambda x : remove_words(x, stop_words_join))
df['Tweet'] = df['Tweet'].str.strip()

In [ ]:
# Explore
df['Tweet'].sample(20).unique()

array(['sipit', 'bodoh nenek',
       'selamat jokowi berhasil menambah hutang bayar rakyat good luck',
       'kabar undang undang md',
       'baju yahudi bertopi bas geng jubah putih kopiah minyak atar berjanggut dahi belekok bak ahli syurga out of place mohon dirajam',
       'khusus pemilihan berharap calon presiden situasi kondisi teramat genting berbicaralah nurani partai politik insya allah ganti presiden rubah kem',
       'video tenaga kerja asing malaysia mengutip pernyataan',
       'pemburu nazi simon wiesenthal memegang foto foto menggambarkan mantan kepala gestapo laki laki rauf van kamar gas bergerak mengeksekusi orang orang yahudi',
       'puasa sok sokan cerita buka kantor ngentot fuck pencitraan yosy bin darto',
       'bertaji komisi pemberantasan korupsi era era komitmen pemberantasan korupsi surat pemberitahuan tahunan era susilo bambang yudhoyono',
       'harga yahudi durian',
       'komunis laku dimana isu jualan antek orde pendukung penerapan negara islam kh

In [ ]:
check('jokowi')

array(['lengserkan orang dasarnya pengecut jokowi',
       'kesempatan presiden menyatn negara ekonomi kuat perkara mudah tantangan ujian dihadapi penuh keyakinan optimisme jokowimasyartoptimistis',
       'pendukung napi ahok jokowi partai keadilan sejahtera kasihan umat cebong',
       'tolak kebangkitan partai komunis indonesia tolak peraturan pemerintah pengganti undang undang anti islam jokowi diktator lengserkan jokowi rakyat tentara nasional indonesia',
       'golkar partai koalisi ajukan cpres jokowi',
       'rejim jokowi otoriter memaksa pakai non tunai',
       'memuji jokowi bu mega takut penjara',
       'tabrn kapal pembohong bangsa indonesia kerja jokowi kodok undercover',
       'menteri badan usaha milik negara dewe miskin prestasi conflict of interest dipertnkan pres jokowi',
       'ditakutkan prabowo sukses jokowi takut prabowo'], dtype=object)

# **Exploration**

### Random Sample

In [ ]:
# Sample HS
df[df['HS'] == 1]['Tweet'].sample(10).unique()

array(['kebanyn duit cong duit partai partai asing asing rampok duit rakyat jual rakyat otak dimana cong',
       'setuju usir buddha indonesia',
       'alah bacot mulut lelaki terkadang', 'buddha kafir teroris',
       'diktator jokowi jongos organisasi jagal manusia nu mengatur siasat kampanye',
       'kafir bodoh bengap terangkan pon pm patut kafir',
       'bolot lemot doang mengerti', 'tolol', 'goblok orang berbeda cong',
       'diam tai anaconda'], dtype=object)

In [ ]:
# Sample non HS
df[df['HS'] == 0]['Tweet'].sample(10).unique()

array(['twitter logonya burung isinya manusia anjing monyet babi',
       'psi sanggah nyatn dukungan pasangan gus ipul puti aliansi santri pemuda ekonom kiai arumi bachsin bagus banyuwangi barisan gus sholatu barisan msantri loyalis khofifah barisan reln khofifah black kampanye calon gubernur',
       'ulama nu selamat ikuti al alquran as sunnah',
       'gembel belr besok ujian nasional', 'wkwk kunyuk jepang nonton rv',
       'lengkap penderitaan pi kontol npi tangan pas pi tangan cewek kesakitan',
       'deklarasi pilihan kepala daerah aman anti hoaks komunitas ojek jabon',
       'dengarkan cerita etnis tionghoa dampak pp keluarganya salah satunya indonesia cina nyaman tionghoa indonesia',
       'bikin fan fiction au menjurus anggota jujur gue menikmati bacanya gue suppolgbt does not mean not normal used to it teman gue terang terangan',
       'pengeruk lelah sanggup mengerok oke disudahi deh dikasih minyak cina finishing selesai beres mengambil gel pakai'],
      dtype=object)

### Unigram

In [ ]:
# Overall Unigram
unigram = df['Tweet'].str.split(expand = True).stack()
unigram.value_counts().head(30)

indonesia    1663
presiden     1416
orang        1369
gue          1248
jokowi       1055
islam         885
agama         763
partai        743
asing         737
komunis       726
rakyat        676
cina          618
negara        556
cebong        540
daerah        517
ganti         481
gubernur      459
kepala        405
kristen       393
kafir         373
kerja         371
ahok          369
rezim         362
ulama         359
tau           338
anak          335
pilihan       334
allah         332
ekonomi       327
prabowo       313
dtype: int64

In [ ]:
# Hatespeech unigram
hs_con = df['HS'] == 1
hs_unigram = df[hs_con]['Tweet'].str.split(expand = True).stack()
hs_unigram.value_counts().head(30)

indonesia     860
presiden      704
jokowi        687
orang         524
cebong        516
partai        486
komunis       474
islam         424
rakyat        415
cina          410
gue           406
ganti         401
ahok          334
agama         284
kafir         267
negara        260
prabowo       246
asing         239
rezim         231
dasar         213
korupsi       195
anjing        189
lengserkan    180
dungu         178
goblok        177
kampret       173
antek         166
bubarkan      164
anies         156
babi          155
dtype: int64

In [ ]:
# Non-hatespeech unigram
non_hs_con = df['HS'] == 0
non_hs_unigram = df[non_hs_con]['Tweet'].str.split(expand = True).stack()
non_hs_unigram.value_counts().head(30)

orang        845
gue          842
indonesia    803
presiden     712
asing        498
agama        479
islam        461
daerah       420
jokowi       368
gubernur     349
kristen      347
kepala       331
pilihan      300
negara       296
budaya       295
ekonomi      287
rakyat       261
partai       257
komunis      252
katolik      248
ulama        242
kerja        219
anak         211
cina         208
tau          202
allah        199
suka         187
bom          186
nusantara    182
yahudi       177
dtype: int64

# **Data Selection**

In [ ]:
data = df[['Tweet', 'HS']]
data.columns = ['tweet','hs']
data.head()

tweet  hs
0  cowok berus melacak perhatian gue lantas remen...   1
1  telat tau edan sarap gue bergaul cigax jifla c...   0
2  kadang berpikir percaya tuhan padl jatuh berka...   0
3                                ku tau matamu sipit   0
4                  kaum cebong kafir dongoknya dungu   1

In [ ]:
data['hs'].value_counts()

0    7608
1    5561
Name: hs, dtype: int64

In [ ]:
data['tweet'] = data['tweet'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13169 entries, 0 to 13168
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   13169 non-null  object
 1   hs      13169 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 308.6+ KB


In [ ]:
# Export to CSV
data.to_csv('/content/drive/MyDrive/Colab Notebooks/hatespeech_dataset.csv', index = False)